In [35]:
import pandas as pd
import numpy as np
import spacy
from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud
%matplotlib inline  

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
df = pd.read_csv("preprocessed_kindle_review[1].csv")

In [37]:
df

,Unnamed: 0,rating,reviewText,summary
0,0,5,This book was the very first bookmobile book I...,50 + years ago...
1,1,1,"When I read the description for this book, I c...",Boring! Boring! Boring!
2,2,5,I just had to edit this review. This book is a...,Wiggleliscious/new toy ready/!!
3,3,5,I don't normally buy 'mystery' novels because ...,Very good read.
4,4,5,"This isn't the kind of book I normally read, a...",Great Story!
...,...,...,...,...
11995,11995,2,Had to read certain passages twice--typos. Wi...,Where's the meat?
11996,11996,3,Not what i expected. yet a very interesting bo...,Interesting
11997,11997,5,Dragon Knights is a world where Knights ride d...,"Dragon Knights, Wings of Change (I Dream of Dr..."
11998,11998,4,"Since this story is very short, it's hard to s...","Good writing, short story"


In [38]:
# if rating is above 3 we will consider it as 1 else 0.

df["rating"] = df["rating"].apply(lambda x: 1 if x < 3  else 0) # positive as 0 and negative as 1
df

,Unnamed: 0,rating,reviewText,summary
0,0,0,This book was the very first bookmobile book I...,50 + years ago...
1,1,1,"When I read the description for this book, I c...",Boring! Boring! Boring!
2,2,0,I just had to edit this review. This book is a...,Wiggleliscious/new toy ready/!!
3,3,0,I don't normally buy 'mystery' novels because ...,Very good read.
4,4,0,"This isn't the kind of book I normally read, a...",Great Story!
...,...,...,...,...
11995,11995,1,Had to read certain passages twice--typos. Wi...,Where's the meat?
11996,11996,0,Not what i expected. yet a very interesting bo...,Interesting
11997,11997,0,Dragon Knights is a world where Knights ride d...,"Dragon Knights, Wings of Change (I Dream of Dr..."
11998,11998,0,"Since this story is very short, it's hard to s...","Good writing, short story"


In [39]:
df.shape

(12000, 4)

In [40]:
df['rating'].value_counts() 

0    8000
1    4000
Name: rating, dtype: int64

In [41]:
df.reviewText = df.reviewText.apply(lambda x: x.lower()) 

In [42]:
! pip install contractions

In [43]:
import contractions

In [44]:
def con(text):
    expand=contractions.fix(text)
    return expand
df.reviewText = df.reviewText.apply(con)
df['reviewText'][0] 

'this book was the very first bookmobile book i bought when i was in the school book club. i loved the story then and i bet a dollar to a donut i will love it again. if my memory serves, i bought this book in 5th grade. that would have been about 1961. i am looking forward to reliving the memories.'

In [45]:
import re
def remove_sp(text):
    pattern=r'[^A-Za-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
df.reviewText=df.reviewText.apply(remove_sp)
df 

,Unnamed: 0,rating,reviewText,summary
0,0,0,this book was the very first bookmobile book i...,50 + years ago...
1,1,1,when i read the description for this book i co...,Boring! Boring! Boring!
2,2,0,i just had to edit this review this book is an...,Wiggleliscious/new toy ready/!!
3,3,0,i do not normally buy mystery novels because i...,Very good read.
4,4,0,this is not the kind of book i normally read a...,Great Story!
...,...,...,...,...
11995,11995,1,had to read certain passages twicetypos wish ...,Where's the meat?
11996,11996,0,not what i expected yet a very interesting boo...,Interesting
11997,11997,0,dragon knights is a world where knights ride d...,"Dragon Knights, Wings of Change (I Dream of Dr..."
11998,11998,0,since this story is very short it is hard to s...,"Good writing, short story"


In [46]:
import string
punctuations=list(string.punctuation)

In [47]:
df.reviewText=df.reviewText.apply(lambda x : " ".join(x for x in x.split()if x not in punctuations)) 

In [48]:
stopword_list=stopwords.words('english')

In [49]:
stopword_list.append('not')

In [50]:
df.reviewText=df.reviewText.apply(lambda x : " ".join(x for x in x.split()if x not in stopword_list)) 

In [51]:
df.reviewText

0        book first bookmobile book bought school book ...
1        read description book could wait read download...
2        edit review book believe got right updated rew...
3        normally buy mystery novels like however time ...
4        kind book normally read although try limit cer...
                               ...                        
11995    read certain passages twicetypos wish built re...
11996    expected yet interesting book usually don8216t...
11997    dragon knights world knights ride dragons slay...
11998    since story short hard say much without giving...
11999    1922 amazing collection info symbols cultures ...
Name: reviewText, Length: 12000, dtype: object

In [52]:
df['reviewText']=df.reviewText.apply(word_tokenize)
df['reviewText'][0] 

['book',
 'first',
 'bookmobile',
 'book',
 'bought',
 'school',
 'book',
 'club',
 'loved',
 'story',
 'bet',
 'dollar',
 'donut',
 'love',
 'memory',
 'serves',
 'bought',
 'book',
 '5th',
 'grade',
 'would',
 '1961',
 'looking',
 'forward',
 'reliving',
 'memories']

In [53]:
lemmatizer=WordNetLemmatizer()

In [54]:
df['reviewText']=df.reviewText.apply(lambda x:[lemmatizer.lemmatize(word) for word in x])

In [55]:
df

,Unnamed: 0,rating,reviewText,summary
0,0,0,"[book, first, bookmobile, book, bought, school...",50 + years ago...
1,1,1,"[read, description, book, could, wait, read, d...",Boring! Boring! Boring!
2,2,0,"[edit, review, book, believe, got, right, upda...",Wiggleliscious/new toy ready/!!
3,3,0,"[normally, buy, mystery, novel, like, however,...",Very good read.
4,4,0,"[kind, book, normally, read, although, try, li...",Great Story!
...,...,...,...,...
11995,11995,1,"[read, certain, passage, twicetypos, wish, bui...",Where's the meat?
11996,11996,0,"[expected, yet, interesting, book, usually, do...",Interesting
11997,11997,0,"[dragon, knight, world, knight, ride, dragon, ...","Dragon Knights, Wings of Change (I Dream of Dr..."
11998,11998,0,"[since, story, short, hard, say, much, without...","Good writing, short story"


In [56]:
df.reviewText=df.reviewText.astype(str)

In [57]:
import numpy as np
reviewText_list = df['reviewText'].array
reviewText_list.shape 

(12000,)

In [58]:
X = df['reviewText'] 
y = df['rating'] 

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)  

In [60]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [61]:
final = Pipeline([('vect',TfidfVectorizer()),
                 ('model',SVC(probability=True))]) 

In [62]:
final.fit(X_train,y_train)

Pipeline(steps=[('vect', TfidfVectorizer()), ('model', SVC(probability=True))])

In [63]:
y_pred = final.predict(X_test)
y_pred

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [64]:
print(classification_report(y_test, y_pred))  

              precision    recall  f1-score   support

           0       0.84      0.93      0.88      2630
           1       0.82      0.66      0.73      1330

    accuracy                           0.84      3960
   macro avg       0.83      0.79      0.81      3960
weighted avg       0.84      0.84      0.83      3960



In [65]:
import pickle

In [66]:
pickle.dump(final,open('sentiment_analysis.p','wb'))